# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 4 2023 11:13AM,254270,15,8988994,"Brookfield Pharmaceuticals, LLC",Released
1,Jan 4 2023 11:08AM,254268,10,MSP217213,"Methapharm, Inc.",Released
2,Jan 4 2023 11:08AM,254268,10,MSP217216,"Methapharm, Inc.",Released
3,Jan 4 2023 11:08AM,254268,10,MSP217218,"Methapharm, Inc.",Released
4,Jan 4 2023 11:07AM,254267,16,TASA-284997,TASA USA Inc.,Released
5,Jan 4 2023 11:01AM,254266,21,637666,"NBTY Global, Inc.",Released
6,Jan 4 2023 10:54AM,254265,16,TASA-284193,TASA USA Inc.,Released
7,Jan 4 2023 10:53AM,254264,20,8968296,"Exact-Rx, Inc.",Released
8,Jan 4 2023 10:53AM,254264,20,8968298,"Exact-Rx, Inc.",Executing
9,Jan 4 2023 10:53AM,254264,20,8968305,"Exact-Rx, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,254265,Released,1
39,254266,Released,1
40,254267,Released,1
41,254268,Released,3
42,254270,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254265,NaN,NaN,1.0
254266,NaN,NaN,1.0
254267,NaN,NaN,1.0
254268,NaN,NaN,3.0
254270,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254157,1.0,0.0,1.0
254171,58.0,49.0,5.0
254172,0.0,3.0,0.0
254187,0.0,0.0,3.0
254190,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254157,1,0,1
254171,58,49,5
254172,0,3,0
254187,0,0,3
254190,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254157,1,0,1
1,254171,58,49,5
2,254172,0,3,0
3,254187,0,0,3
4,254190,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254157,1,,1
1,254171,58,49,5
2,254172,,3,
3,254187,,,3
4,254190,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 4 2023 11:13AM,254270,15,"Brookfield Pharmaceuticals, LLC"
1,Jan 4 2023 11:08AM,254268,10,"Methapharm, Inc."
4,Jan 4 2023 11:07AM,254267,16,TASA USA Inc.
5,Jan 4 2023 11:01AM,254266,21,"NBTY Global, Inc."
6,Jan 4 2023 10:54AM,254265,16,TASA USA Inc.
7,Jan 4 2023 10:53AM,254264,20,"Exact-Rx, Inc."
14,Jan 4 2023 10:45AM,254263,12,Livs Products
15,Jan 4 2023 10:42AM,254262,16,TASA USA Inc.
16,Jan 4 2023 10:42AM,254261,10,"Methapharm, Inc."
23,Jan 4 2023 10:40AM,254260,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 4 2023 11:13AM,254270,15,"Brookfield Pharmaceuticals, LLC",,,1
1,Jan 4 2023 11:08AM,254268,10,"Methapharm, Inc.",,,3
2,Jan 4 2023 11:07AM,254267,16,TASA USA Inc.,,,1
3,Jan 4 2023 11:01AM,254266,21,"NBTY Global, Inc.",,,1
4,Jan 4 2023 10:54AM,254265,16,TASA USA Inc.,,,1
5,Jan 4 2023 10:53AM,254264,20,"Exact-Rx, Inc.",,3,4
6,Jan 4 2023 10:45AM,254263,12,Livs Products,,,1
7,Jan 4 2023 10:42AM,254262,16,TASA USA Inc.,,,1
8,Jan 4 2023 10:42AM,254261,10,"Methapharm, Inc.",,,7
9,Jan 4 2023 10:40AM,254260,10,Emerginnova,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 11:13AM,254270,15,"Brookfield Pharmaceuticals, LLC",1,,
1,Jan 4 2023 11:08AM,254268,10,"Methapharm, Inc.",3,,
2,Jan 4 2023 11:07AM,254267,16,TASA USA Inc.,1,,
3,Jan 4 2023 11:01AM,254266,21,"NBTY Global, Inc.",1,,
4,Jan 4 2023 10:54AM,254265,16,TASA USA Inc.,1,,
5,Jan 4 2023 10:53AM,254264,20,"Exact-Rx, Inc.",4,3,
6,Jan 4 2023 10:45AM,254263,12,Livs Products,1,,
7,Jan 4 2023 10:42AM,254262,16,TASA USA Inc.,1,,
8,Jan 4 2023 10:42AM,254261,10,"Methapharm, Inc.",7,,
9,Jan 4 2023 10:40AM,254260,10,Emerginnova,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 11:13AM,254270,15,"Brookfield Pharmaceuticals, LLC",1,,
1,Jan 4 2023 11:08AM,254268,10,"Methapharm, Inc.",3,,
2,Jan 4 2023 11:07AM,254267,16,TASA USA Inc.,1,,
3,Jan 4 2023 11:01AM,254266,21,"NBTY Global, Inc.",1,,
4,Jan 4 2023 10:54AM,254265,16,TASA USA Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 11:13AM,254270,15,"Brookfield Pharmaceuticals, LLC",1.0,NaN,NaN
1,Jan 4 2023 11:08AM,254268,10,"Methapharm, Inc.",3.0,NaN,NaN
2,Jan 4 2023 11:07AM,254267,16,TASA USA Inc.,1.0,NaN,NaN
3,Jan 4 2023 11:01AM,254266,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jan 4 2023 10:54AM,254265,16,TASA USA Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 11:13AM,254270,15,"Brookfield Pharmaceuticals, LLC",1.0,0.0,0.0
1,Jan 4 2023 11:08AM,254268,10,"Methapharm, Inc.",3.0,0.0,0.0
2,Jan 4 2023 11:07AM,254267,16,TASA USA Inc.,1.0,0.0,0.0
3,Jan 4 2023 11:01AM,254266,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jan 4 2023 10:54AM,254265,16,TASA USA Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2796613,47.0,7.0,0.0
12,254263,1.0,0.0,0.0
15,1271023,105.0,61.0,59.0
16,1017046,4.0,0.0,0.0
19,2796421,81.0,14.0,0.0
20,508501,4.0,4.0,0.0
21,508476,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2796613,47.0,7.0,0.0
1,12,254263,1.0,0.0,0.0
2,15,1271023,105.0,61.0,59.0
3,16,1017046,4.0,0.0,0.0
4,19,2796421,81.0,14.0,0.0
5,20,508501,4.0,4.0,0.0
6,21,508476,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,47.0,7.0,0.0
1,12,1.0,0.0,0.0
2,15,105.0,61.0,59.0
3,16,4.0,0.0,0.0
4,19,81.0,14.0,0.0
5,20,4.0,4.0,0.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,47.0
1,12,Released,1.0
2,15,Released,105.0
3,16,Released,4.0
4,19,Released,81.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,59.0,0.0,0.0,0.0,0.0
Executing,7.0,0.0,61.0,0.0,14.0,4.0,1.0
Released,47.0,1.0,105.0,4.0,81.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,59.0,0.0,0.0,0.0,0.0
1,Executing,7.0,0.0,61.0,0.0,14.0,4.0,1.0
2,Released,47.0,1.0,105.0,4.0,81.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,59.0,0.0,0.0,0.0,0.0
1,Executing,7.0,0.0,61.0,0.0,14.0,4.0,1.0
2,Released,47.0,1.0,105.0,4.0,81.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()